In [8]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('C:/Users/vitor/3D Objects/Mestrado/Simulacao/Cubebot/cameraCalib/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(0)
cv.destroyAllWindows()

In [28]:
import cv2
import numpy as np
import pathlib

#amostras coletadas em um tabuleiro 8x8 com casas de 1,95cm
pasta = "/cameraCalib/"
extensao = ".jpg"
tamQuad = 1.95
larg = 7
alt = 6

def calibrate_chessboard(dir_path, image_format, square_size, width, height):
    '''Calibrate a camera using chessboard images.'''
    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    #images = pathlib.Path(dir_path).glob(f'*.{image_format}')
    images = glob.glob('C:/Users/vitor/3D Objects/Mestrado/Simulacao/Cubebot/cameraCalib/*.jpg')
    # Iterate through all images
    for fname in images:
        #img = cv2.imread(str(fname))
        img = cv.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), None)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return [ret, mtx, dist, rvecs, tvecs]

In [29]:
intrinsic = calibrate_chessboard(pasta,extensao,tamQuad,larg,alt)

In [27]:
intrinsic[1]

array([[1.18895786e+03, 0.00000000e+00, 3.34052137e+02],
       [0.00000000e+00, 1.19015126e+03, 2.57255437e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [31]:
IM = intrinsic[1]
fx = IM[0][0]
fx

1188.9588691371891

In [32]:
fy = IM[1][1]
f = (fx+fy)/2
f

1189.5555738648386

In [39]:
c_u = IM[0][2]
c_v = IM[1][2]
imAlpha = fy/fx 
imAlpha

1.0010037432633516

In [40]:
c_u

334.05207993831146

In [41]:
c_v

257.25560740793105

In [45]:
def tImagem(x_im, y_im, z_real):
    c_u = 334.052
    c_v = 257.256
    imAlpha = 1.001
    f = 1189.5555738648386
    x_real = (z_real*(x_im-c_u))/(f*imAlpha)
    y_real = (z_real*(y_im-c_v))/(f)
    return([x_real,y_real,z_real])

In [48]:
tImagem(0,0,980)

[-274.9294974268062, -211.93703391334424, 980]